In [1]:
import pandas as pd
from tqdm import tqdm
import sys
sys.path.append("..")
from src.funcs import *

In [2]:
# async def get_cat_links(session, url):
#     categorias_url = []
#     async with session.get(url) as response:
#         html = await response.text()
#         soup = BeautifulSoup(html, 'html.parser')
#         for categoria in soup.findAll('div', class_="card h-100"):
#             categorias_url.append(categoria.find('a').get("href"))
#     return categorias_url

# async def main(urls):
#     async with aiohttp.ClientSession() as session:
#         tasks = [get_cat_links(session, url) for url in urls]
#         list_cats = await asyncio.gather(*tasks)
#     return list_cats

# urls = links_list
# await main(urls)

In [3]:
supermercados = ["mercadona", "carrefour", "eroski", "dia", "hipercor", "alcampo"]

In [4]:
superurls = [f'https://super.facua.org/{s}/' for s in supermercados]

In [5]:
dfs_super_list = []
supermercados = ["mercadona", "carrefour", "eroski", "dia", "hipercor", "alcampo"]
superurls = [f'https://super.facua.org/{s}/' for s in supermercados]
for i in tqdm(range(len(supermercados))):
    dfs_cat_list = []
    urls_cats, cats = get_cats(superurls[i])
    for url2, cat in zip(urls_cats, cats):
        # print("url2")
        dfs_subcat_list = []
        urls_subcat, subcats = get_cats(url2)
        if not urls_subcat and not subcats:
            urls_hist = get_hist(url2)
            dfs_hist_list = []
            for url3 in urls_hist:
                # print("url3")
                df_producto = get_tab(url3)
                dfs_hist_list.append(df_producto)
            df_hist = pd.concat(dfs_hist_list)
            df_hist["Subcategoria"] = None
            dfs_subcat_list.append(df_hist)
        else:
            for url3, subcat in zip(urls_subcat, subcats):
                # print("url3")
                dfs_hist_list = []
                urls_hist = get_hist(url3)
                for url4 in urls_hist:
                    # print("url4")
                    df_producto = get_tab(url4)
                    dfs_hist_list.append(df_producto)
                df_hist = pd.concat(dfs_hist_list)
                df_hist["Subcategoria"] = subcat
                dfs_subcat_list.append(df_hist)
                # print("helo\n", dfs_subcat_list)
            # print("out")
        df_subcat = pd.concat(dfs_subcat_list)
        df_subcat["Categoria"] = cat
        dfs_cat_list.append(df_subcat)
    df_cat = pd.concat(dfs_cat_list)
    df_cat["Supermercado"] = supermercados[i]
    dfs_super_list.append(df_cat)    

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [03:52<00:00, 38.75s/it]


In [6]:
df_complete = pd.concat(dfs_super_list)

In [7]:
df_complete["Precio (€)"] = df_complete["Precio (€)"].str.replace(",",".").astype(float)

In [8]:
df_complete = pd.concat([df_complete,df_complete["Variación"].str.replace("=","0").str.replace(",",".").str.replace("(","").str.replace(")","").str.replace("%","").str.split(expand=True).fillna("0")], axis=1)

In [9]:
df_complete = df_complete.rename({0: 'abs_variacion', 1: 'precent_variacion'}, axis=1).drop(columns='Variación')

In [10]:
df_complete.columns

Index(['Día', 'Precio (€)', 'Nombre', 'Subcategoria', 'Categoria',
       'Supermercado', 'abs_variacion', 'precent_variacion'],
      dtype='object')

In [11]:
columnas = df_complete.columns
orden = [2, 0, 1, -2, -1, 3, 4, 5]
columnas_reordenadas = [columnas[i] for i in orden]

In [12]:
df_complete = df_complete.reindex(columns=columnas_reordenadas).reset_index(drop=True)

In [13]:
df_complete["abs_variacion"] = df_complete["abs_variacion"].astype(float)
df_complete["precent_variacion"] = df_complete["precent_variacion"].astype(float)

In [14]:
df_complete["Día"] = df_complete["Día"].map(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

In [17]:
df_complete.to_csv('../datos/df_complete.csv')